In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [5]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load prompt_template.md

In [6]:
sys.path.append('..')
from utils import *
from functools import partial

In [7]:
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [ ]:
prompt_dict().keys()

# Prompt fill place holder

In [9]:
import json

In [10]:
with open('2108.02159.jsonl','r') as f:
    kwargs_jsonl= [json.loads(line) for line in f]

In [11]:
def load_excerpt(sources):
    excerpt=''
    for tex, lines in sources.items():
        with open(tex,'r') as f:
            f_list=list(f)
            for line in lines:
                excerpt=excerpt+''.join(f_list[line[0]:line[1]])
    return excerpt

In [12]:
prompt_system='''I will provide you a Excerpt of physics paper, and a Template. You will need to fill the placeholders in the template using the correct information from the excerpt.
Here are conventions: 
{..} means a placeholder which you need to fill by extracting information from the excerpt.
{A|B} means you need to make a choice between A and B
[..] means optional sentence. You should decide whether to use it depending on the excerpt.
{{..}} DOES NOT mean a placeholder. You should not change the content inside double curly braces {{..}}.
`You should recall that {expression_second_nonint}.` : this sentence should be kept as is.


Finally, if you cannot figure out the placeholder, you should leave it as is.'''

In [13]:
def drop_text_after(string):
    string_new=''
    for line in string.split('\n'):
        if '===' in line:
            break
        else:
            if 'You should recall that' not in line:
                string_new='\n'.join([string_new,line])
    return string_new[1:]


In [14]:
def extractor(descriptor,message=False):
    sys_msg=[{'role': 'system', 'content': prompt_system}]
    question_prompt='\nTemplate:\n {template} \n\n Excerpt:\n {excerpt}'.format(template=drop_text_after(prompt_dict()[descriptor['task']]), excerpt=load_excerpt(descriptor['source']))
    user_msg=[{'role':'user','content':question_prompt}]
    messages = sys_msg + user_msg
    if message:
        return sys_msg[0]['content']+user_msg[0]['content']
    rs= openai.ChatCompletion.create(messages=messages, **model_params)
    response=rs['choices'][0]['message'].content
    return response


In [15]:
print('# '+kwargs_jsonl[0]['task'])

# Construct Kinetic Hamiltonian (continuum version, second-quantized)


In [16]:
response=(extractor(kwargs_jsonl[0]))

In [17]:
print(response)

You will be instructed to describe the kinetic term of Hamiltonian in TMD moiré materials in the real space in the single-particle form.   
The degrees of freedom of the system are: spin/valley.  
Express the Kinetic Hamiltonian $T$ using $\hbar$, $m^*$, $\b{k}$, and $\b{b}$, where the summation of $\b{k}$ should be running over the first Brillouin zone.

Use the following conventions for the symbols:  
$\hbar$ is the reduced Planck constant, $m^*$ is the effective mass of the electron, $\b{k}$ is the momentum in the first moiré Brillouin zone, and $\b{b}$'s are moiré reciprocal lattice vectors.
